In [ ]:
import numpy as np
import pandas as pd
import os
import pydicom

In [ ]:
train_dir = "../input/osic-pulmonary-fibrosis-progression/train"
test_dir = "../input/osic-pulmonary-fibrosis-progression/test"

In [ ]:
ID_dir_list = os.listdir(train_dir)
len(ID_dir_list)

In [ ]:
eliminated_ID_list = []
df_list=[]

for ID_dir in ID_dir_list:
    df_base = pd.DataFrame(columns=["name","value"])
    files = os.listdir(os.path.join(train_dir,ID_dir))
    for f in files:
        ds = pydicom.dcmread(os.path.join(train_dir,os.path.join(ID_dir,f)))
        df_temp = pd.DataFrame(ds.values())
        if df_temp.shape[1] >1:
            print(ID_dir+" is eliminated.")
            eliminated_ID_list.append(ID_dir)
            break
        df_temp[0] = df_temp[0].apply(lambda x: pydicom.dataelem.DataElement_from_raw(x) 
                                if isinstance(x, pydicom.dataelem.RawDataElement) else x)
        df_temp['name'] = df_temp[0].apply(lambda x: x.name)
        df_temp['value'] = df_temp[0].apply(lambda x: x.value)
        df_temp = df_temp.drop(df_temp.index[df_temp["name"].str.contains("Pixel Data")])
        df_val = df_temp.drop(0,axis=1)
        df_ID_dcmnum = pd.DataFrame([["ID",ID_dir],["dcm_num",f]],columns= ['name','value'])
        df_add = df_ID_dcmnum.append(df_val)
        df_base = pd.merge(df_base,df_add,on="name",how="outer")
    df_base=df_base.T
    df_fix = df_base[2:]
    df_fix.columns = df_base.iloc[1].to_list()
    df_list.append(df_fix)


In [ ]:
df_final = pd.concat(df_list)
df_final = df_final.reset_index()
df_final = df_final.drop("index",axis=1)
df_final.to_csv("dcm_info_train.csv",index=False)

In [ ]:
eliminated_df_list=[]

for ID_dir in eliminated_ID_list:
    df_base = pd.DataFrame(columns=["name","value"])
    files = os.listdir(os.path.join(train_dir,ID_dir))
    for f in files:
        ds = pydicom.dcmread(os.path.join(train_dir,os.path.join(ID_dir,f)))
        print(ds)
        df_temp = pd.DataFrame(ds.values())
        if df_temp.shape[1] >1:
            print(ID_dir+" is eliminated.")
            break
        df_temp[0] = df_temp[0].apply(lambda x: pydicom.dataelem.DataElement_from_raw(x) 
                                if isinstance(x, pydicom.dataelem.RawDataElement) else x)
        df_temp['name'] = df_temp[0].apply(lambda x: x.name)
        df_temp['value'] = df_temp[0].apply(lambda x: x.value)
        df_temp = df_temp.drop(df_temp.index[df_temp["name"].str.contains("Pixel Data")])
        df_val = df_temp.drop(0,axis=1)
        df_ID_dcmnum = pd.DataFrame([["ID",ID_dir],["dcm_num",f]],columns= ['name','value'])
        df_add = df_ID_dcmnum.append(df_val)
        df_base = pd.merge(df_base,df_add,on="name",how="outer")
    print(ID_dir+" is passed.")
    df_base=df_base.T
    df_fix = df_base[2:]
    df_fix.columns = df_base.iloc[1].to_list()
    eliminated_df_list.append(df_fix)


In [ ]:
df_final_2 = pd.concat(eliminated_df_list)
df_final_2 = df_final_2.reset_index()
df_final_2 = df_final_2.drop("index",axis=1)

In [ ]:
df_train_dcm_info  = pd.concat([df_final,df_final_2],axis=0)
df_train_dcm_info.to_csv("dcm_info_train.csv",index=False)